<a href="https://colab.research.google.com/github/cph316/generative_ai_for_humanities/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C_v9_%E8%A9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 📌 安裝必要套件
!pip install openai
!pip install openai folium gradio
!pip install openai requests
# 🚀下載 stabilityai/stable-diffusion-2-1 模型
# 直接在本地模型生成圖片
!pip install diffusers accelerate transformers safetensors
!pip install torch torchvision torchaudio
!pip install fpdf pillow
!pip install fpdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from PIL import Image
import gradio as gr
import requests
import time
import torch
from fpdf import FPDF
from io import BytesIO
# Enable deprecation warnings:
import warnings

from base64 import b64encode
from IPython.display import display, HTML

/usr/local/lib/python3.11/dist-packages/fpdf/__init__.py:40: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [ ]:
# 📌 Groq 登入
for var in ["PROVIDER", "GROQ_API_KEY", "OPENAI_API_KEY"]:
    if var in os.environ:
        os.environ.pop(var)
        print(f"🔄 已清空環境變數: {var}")

# 輸入 Groq API 金鑰
os.environ["GROQ_API_KEY"] = input("請輸入 Groq API 金鑰 (gsk- 開頭)：").strip()

api_key = os.environ.get("GROQ_API_KEY")
model = "meta-llama/llama-4-scout-17b-16e-instruct"
api_url = "https://api.groq.com/openai/v1/chat/completions"

# 檢查金鑰是否存在
if not api_key:
    raise ValueError("❌ 找不到 Groq API 金鑰，請先設定環境變數！")

print(f"✅ 使用模型：GROQ ({model})")
print(f"✅ API Endpoint：{api_url}")

🔄 已清空環境變數: GROQ_API_KEY
請輸入 Groq API 金鑰 (gsk- 開頭)：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
✅ 使用模型：GROQ (meta-llama/llama-4-scout-17b-16e-instruct)
✅ API Endpoint：https://api.groq.com/openai/v1/chat/completions
🔄 已清空環境變數: GROQ_API_KEY
請輸入 Groq API 金鑰 (gsk- 開頭)：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
✅ 使用模型：GROQ (meta-llama/llama-4-scout-17b-16e-instruct)
✅ API Endpoint：https://api.groq.com/openai/v1/chat/completions


In [ ]:
# ✅ Groq API 金鑰驗證

def check_groq_api_key(api_key, model="meta-llama/llama-4-scout-17b-16e-instruct"):
    api_url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": "Say hello"}],
        "max_tokens": 5
    }

    try:
        response = requests.post(api_url, headers=headers, json=payload, timeout=10)
        if response.status_code == 200:
            print("✅ Groq API 金鑰驗證成功！可以正常使用！")
            print("✅ 測試回應:", response.json()["choices"][0]["message"]["content"])
            return True
        else:
            print(f"❌ Groq API 錯誤！狀態碼: {response.status_code}")
            print("錯誤訊息:", response.text)
            return False
    except Exception as e:
        print(f"❌ 發生例外錯誤: {e}")
        return False

# 金鑰驗證
check_groq_api_key(api_key)

✅ Groq API 金鑰驗證成功！可以正常使用！
✅ 測試回應: Hello! How can I


True

✅ Groq API 金鑰驗證成功！可以正常使用！
✅ 測試回應: Hello! How are you


True

In [ ]:
# ✅ LLM (Groq API)回應
def llm_reply(prompt, chat_history=None):
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    # 支援多輪聊天，傳遞過去歷史訊息
    messages = chat_history if chat_history else []
    messages.append({"role": "user", "content": prompt})
    body = {
        "model": model,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 300
    }
    try:
        response = requests.post(api_url, headers=headers, json=body)
        if response.status_code == 200:
            reply = response.json()["choices"][0]["message"]["content"]
            print(f"✅ LLM 回應: {reply}")
            messages.append({"role": "assistant", "content": reply})
            return reply, messages
        else:
            print(f"❌ Groq API 錯誤: {response.status_code} - {response.text}")
            return f"❌ Groq API 錯誤: {response.status_code}", messages
    except Exception as e:
        print(f"❌ 發生例外錯誤: {e}")
        return f"❌ 發生例外錯誤: {e}", messages

In [ ]:
# 📌 Diffusion 模型初始化
pipe = None

def load_diffusion_model():
    global pipe
    if pipe is None:
        print("⏳ 正在加載 Diffusion 模型...")
        model_id = "stabilityai/stable-diffusion-2-1"
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            scheduler=scheduler,
            torch_dtype=torch.float16
        )
        pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
        print("✅ 模型加載完成！")
    else:
        print("✅ 模型已經加載，直接使用！")
    return pipe

In [ ]:
STYLE_PRESETS = {
    "手繪風": "hand-drawn style, illustration, warm color palette, soft warm tones, high detail",
    "日系可愛": "anime style, cute, Japanese art, pastel colors, soft light",
    "寫實風": "realistic style, high detail, cinematic lighting, 4k resolution",
    "像素風": "pixel art, 8-bit style, retro game aesthetics, bright colors",
    "水彩風": "watercolor painting, soft edges, delicate brush strokes, muted colors"
}

In [ ]:
# 📌 Diffusion 圖像生成
DEFAULT_STYLE_KEY = "手繪風"

def generate_cover_image(prompt, style_choice, default_key=DEFAULT_STYLE_KEY):
    STYLE_PRESETS = {
        "手繪風": "hand-drawn style, illustration, warm color palette, soft warm tones, high detail",
        "日系可愛": "anime style, cute, Japanese art, pastel colors, soft light",
        "寫實風": "realistic style, high detail, cinematic lighting, 4k resolution",
        "像素風": "pixel art, 8-bit style, retro game aesthetics, bright colors",
        "水彩風": "watercolor painting, soft edges, delicate brush strokes, muted colors"
    }
    try:
        load_diffusion_model()
        if style_choice in STYLE_PRESETS:
            style = STYLE_PRESETS[style_choice]
            print(f"🎨 使用者選擇風格：「{style_choice}」")
        else:
            style = STYLE_PRESETS[default_key]
            print(f"🎨 使用者未選擇風格或選擇無效，已套用預設風格：「{default_key}」")
        full_prompt = f"travel guide cover, {prompt}, {style}"
        print(f"🎨 開始生成圖片：{full_prompt}")
        start_time = time.time()
        result = pipe(full_prompt, num_inference_steps=30, guidance_scale=7.5)
        image = result.images[0]
        duration = time.time() - start_time
        print(f"✅ 圖片生成完成，用時 {duration:.1f} 秒")
        return image
    except Exception as e:
        print(f"❌ 發生例外錯誤：{e}")
        return Image.new("RGB", (512, 512), color="gray")

In [ ]:
# 📌 AI Agents - Reflection & Planning Prompt
def agent_plan_route(location, preference, budget, days, group, transport, season="自動推斷"):
    # Reflection + Planning Prompt
    prompt = (
        f"你是資深旅遊規劃AI，具備專業知識與即時判斷能力。\n"
        f"【旅遊資訊】\n"
        f"地點：{location}\n"
        f"偏好：{preference}\n"
        f"預算：{budget}\n"
        f"天數：{days}\n"
        f"人數/身份：{group}\n"
        f"交通方式：{transport}\n"
        f"時節/季節：{season}\n"
        f"請針對上述需求，進行：\n"
        f"1. 完整行程自動規劃，並列出每日路線\n"
        f"2. Reflection：檢查有無「明顯不合理」安排（如冬天安排賞櫻、交通中斷等），自動修正，並說明調整理由\n"
        f"3. 行程包含天氣/景點/交通/餐飲等推薦，禁止出現無法實現的內容\n"
        f"4. 以條列式輸出"
    )
    result, _ = llm_reply(prompt)
    return result

顯示地圖

In [ ]:
from geopy.geocoders import Nominatim
from IPython.display import display, HTML
import folium
from PIL import Image

def generate_map_html(location_name):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(location_name)
    if location:
        m = folium.Map(location=[location.latitude, location.longitude], zoom_start=13)
        folium.Marker([location.latitude, location.longitude], popup=f"{location_name}").add_to(m)
        # m.save('map.html')   # 存成HTML檔案
        # hti = Html2Image()
        # hti.screenshot(html_file='map.html', save_as='map.png')  # map.png 就是地圖截圖
        # map_img = Image.open('map.png')

        return m._repr_html_()
    else:
        return "⚠️ 找不到地點，請確認輸入是否正確"


In [ ]:
generate_map_html("東京")

'<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;html&gt;\n&lt;head&gt;\n    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-3.7.1.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css&quot;/&gt;\n    &lt;link rel=&quot;styles

#PDF

In [ ]:
#加圖片版(目前用這個)
# -*- coding: utf-8 -*-
# 下載支援中文字型的 Noto Sans CJK 字型檔（Regular 與 Bold）
!wget -O NotoSansCJKtc-Regular.ttf "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf"
!wget -O NotoSansCJKtc-Bold.otf "https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Bold.otf"

# 定義產生 PDF 並內嵌圖片與文字內容的函式
def generate_and_display_pdf(image, text="hello world", filename="fpdf2-demo.pdf", font_size=14, width=800, height=400, font_path="/content/NotoSansCJKtc-Regular.ttf"):
    # 匯入所需模組
    import warnings
    from fpdf import FPDF  # 用來產生 PDF
    from base64 import b64encode  # 用來將 PDF 轉成 base64 字串以便 HTML 下載
    from IPython.display import display, HTML  # 用於 Jupyter Notebook 內嵌 HTML
    import io  # 用於圖片轉為記憶體檔案

    # 顯示 Python 的棄用警告（有助於除錯）
    warnings.simplefilter('default', DeprecationWarning)

    # 建立 PDF 文件物件
    pdf = FPDF()
    pdf.add_page()  # 加入一頁

    # 註冊並使用中文字型
    pdf.add_font('NotoSansCJKtc', '', font_path, uni=True)  # Regular
    pdf.set_font('NotoSansCJKtc', size=font_size)  # 預設字型
    pdf.add_font('NotoSansCJKtc', 'B', 'NotoSansCJKtc-Bold.otf')  # Bold
    pdf.set_font('NotoSansCJKtc', style='B', size=font_size)  # 設定為粗體

    # ===== 圖片頁首處理 =====
    image_buffer = io.BytesIO()  # 建立記憶體檔案物件
    image.save(image_buffer, format="PNG")  # 將 PIL 圖片存為 PNG 格式
    image_buffer.seek(0)  # 將檔案指標移至開頭
    pdf.image(image_buffer, x=10, y=20, w=pdf.w - 20)  # 將圖片插入 PDF 頁面
    pdf.ln(100)  # 留空行距避免文字與圖片重疊

    # ===== 處理後續文字內容 =====
    pdf.add_page()  # 加入一頁
    pdf.set_font('NotoSansCJKtc', '', font_size)  # 恢復為 regular 字型
    safe_width = pdf.w - 2 * pdf.l_margin  # 可用文字寬度（扣掉左右邊界）

    # 將輸入文字逐行處理
    for line in text.split("\n"):
        line = line.rstrip()  # 移除右邊空白

        if not line:  # 空行：換段落
            pdf.ln(font_size//2 + 2)

        elif line.startswith('### ') or line.startswith('#### '):  # 標題格式
            pdf.set_font('NotoSansCJKtc', size=font_size+4, style='B')
            pdf.multi_cell(safe_width, font_size+10, line[4:])  # 切除 ###
            pdf.set_font('NotoSansCJKtc', size=font_size)

        elif line.startswith('**') and line.endswith('**'):  # 粗體行
            pdf.set_font('NotoSansCJKtc', size=font_size+2, style='B')
            pdf.multi_cell(safe_width, font_size+6, line.replace('**',''))
            pdf.set_font('NotoSansCJKtc', size=font_size)

        elif line.startswith('*'):  # 列表項目
            pdf.set_x(pdf.l_margin + 10)  # 稍微縮排
            pdf.multi_cell(safe_width-10, font_size, line[1:].strip())

        else:  # 一般段落
            if '**' in line:
                segments = []
                while '**' in line:
                    pre, rest = line.split('**', 1)
                    if '**' not in rest:
                        segments.append((pre + '**' + rest, False))  # 沒有配對，當作普通字
                        break
                    bold, line = rest.split('**', 1)
                    segments.append((pre, False))
                    segments.append((bold, True))
                segments.append((line, False))

                line_height = font_size + 1
                max_width = safe_width
                curr_width = 0
                for seg_text, is_bold in segments:
                    seg_text = seg_text.strip()
                    if not seg_text:
                        continue
                    pdf.set_font('NotoSansCJKtc', style='B' if is_bold else '', size=font_size)
                    # 中文計算寬度
                    text_width = pdf.get_string_width(seg_text)
                    # 斷行（超出就跳行）
                    if curr_width + text_width > max_width:
                        pdf.ln(line_height)
                        pdf.ln(line[curr_width+curr_width,:])
                        pdf.set_x(pdf.l_margin + 5)
                        curr_width = 0
                    pdf.cell(text_width, line_height, seg_text, ln=0, align='L')
                    curr_width += text_width
                pdf.ln(line_height)
                #     text_width = pdf.get_string_width(seg_text)
                #     pdf.cell(text_width, line_height, seg_text, ln=0, align='L')
                # pdf.ln(line_height)  # 一整行結束再換行



                # for seg_text, is_bold in segments:
                #     seg_text = seg_text.strip()
                #     if not seg_text:
                #        continue
                #     pdf.set_font('NotoSansCJKtc', style='B' if is_bold else '', size=font_size)
                #     pdf.set_x(pdf.l_margin + 5)  # 稍微縮排
                #     pdf.multi_cell(safe_width - 20, font_size + 2, seg_text, align='L')
            else:
                pdf.set_font('NotoSansCJKtc', style='', size=font_size)
                pdf.set_x(pdf.l_margin + 5)  # 稍微縮排
                pdf.multi_cell(safe_width, font_size + 2, line, align='L')
            # pdf.set_x(pdf.l_margin + 5)
            # pdf.multi_cell(safe_width-20, font_size+2, line, align='L')

    # 將 PDF 轉為 byte 並編碼成 base64 字串
    pdf_bytes = pdf.output()
    base64_pdf = b64encode(pdf_bytes).decode("utf-8")

    # 回傳 HTML 的下載連結
    html_download = f'<a download="{filename}" href="data:application/pdf;base64,{base64_pdf}">下載 PDF</a>'
    return html_download


--2025-06-03 05:09:25--  https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-03 05:09:25--  https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-03 05:09:25--  https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.19

In [ ]:
# 設定 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")

    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題", placeholder="東京鐵塔、京都櫻花")
            preference = gr.Textbox(label="旅遊偏好", placeholder="自然景觀/人文藝術/美食/購物")
            budget = gr.Textbox(label="預算（可選）", placeholder="例如：2萬元以內")
            days = gr.Textbox(label="天數（可選）", placeholder="例如：5天4夜")
            group = gr.Textbox(label="人數/身份（可選）", placeholder="2人情侶/全家出遊")
            transport = gr.Textbox(label="交通方式（可選）", placeholder="自駕/大眾運輸")
            style_choice = gr.Radio(
                choices=list(STYLE_PRESETS.keys()),
                value=DEFAULT_STYLE_KEY,
                label="圖像風格（請選擇一項）"
            )
            #pdf_name_input = gr.Textbox(label="PDF 檔名", placeholder="例如 ai_travel_plan.pdf") ##
        with gr.Row():
            submit = gr.Button("生成旅遊建議、圖像、地圖與 PDF")
        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示") # ⬅ 新增地圖顯示欄位
        pdf_download = gr.HTML(label="下載 PDF")
        error_output = gr.Textbox(label="錯誤訊息", lines=3)

        def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice, pdf_name_input="cd.pdf"):
            try:
                location="大阪"
                preference="美食"
                budget = "2萬"
                days = "5天4夜"
                group = "1人"
                transport= "自駕"
                user_prompt = (
                f"請根據以下資訊規劃個人化旅遊建議，包含路線/景點/活動。\n"
                f"地點：{location}\n"
                f"旅遊偏好：{preference}\n"
                f"預算：{budget}\n"
                f"天數：{days}\n"
                f"人數或身份：{group}\n"
                f"交通方式：{transport}\n"
                f"請條列化建議內容。"
                )
                # 1) LLM 文字建議
                llm_result, _ = llm_reply(user_prompt)

                # 2) 生成封面圖
                image_result = generate_cover_image(location, style_choice)

                # 3) AI Agents (Reflection + Planning)
                agent_result = agent_plan_route(location, preference, budget, days, group, transport)

                # 4) 用你的 generate_map_html 函數生成地圖
                map_html = generate_map_html(location)  # ⬅ 加入地圖

                # 5) PDF
                pdf_text = llm_result + agent_result
                PP = generate_and_display_pdf(image=image_result ,text=pdf_text, filename="AI_Travel_Plan.pdf", font_size=14, width=800, height=400)

                return llm_result, image_result, agent_result, map_html, PP
            except Exception as e:
                # 發生錯誤時回傳錯誤訊息，其它欄位設為空或 None
                return "123", None, "", "", None, str(e)



        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice],
            outputs=[llm_output, image_output, agent_output, map_display, pdf_download]
        )

    with gr.Tab("互動式旅遊聊天機器人"):
        chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")
        chat_input = gr.Textbox(label="請輸入你的問題", placeholder="請問大阪春天有什麼活動？")
        chat_submit = gr.Button("發送")
        chat_state = gr.State([])  # List of messages

        def chat_ai(user_msg, chat_history):
            reply, new_history = llm_reply(user_msg, chat_history)
            chat_history.append((user_msg, reply))
            return chat_history, new_history

        chat_submit.click(
            fn=chat_ai,
            inputs=[chat_input, chat_state],
            outputs=[chatbox, chat_state]
        )

    demo.launch()
#llm_result, image_result, agent_result, map_html, pdf_path = ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice)

<ipython-input-37-54190f1404c6>:76: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ef35acae8da81f60a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#copy版


In [ ]:
#處理輸入PDF檔名的參考code
                # 處理 PDF 檔名
                if not pdf_filename:
                    pdf_filename = "ai_travel_plan.pdf"
                elif not pdf_filename.lower().endswith(".pdf"):
                    pdf_filename += ".pdf"
                # 5) 匯出 PDF
                # full_text = llm_result + "\n\n" + agent_result
                # pdf_path = save_to_pdf(full_text, image_result, pdf_path=pdf_filename)
                回傳正常結果，錯誤訊息留空

In [ ]:
#家圖片版(目前用這個)
# -*- coding: utf-8 -*-
# 下載支援中文字型的 Noto Sans CJK 字型檔（Regular 與 Bold）
!wget -O NotoSansCJKtc-Regular.ttf "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf"
!wget -O NotoSansCJKtc-Bold.otf "https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Bold.otf"

# 定義產生 PDF 並內嵌圖片與文字內容的函式
def generate_and_display_pdf(image, text="hello world", filename="fpdf2-demo.pdf", font_size=14, width=800, height=400, font_path="/content/NotoSansCJKtc-Regular.ttf"):
    # 匯入所需模組
    import warnings
    from fpdf import FPDF  # 用來產生 PDF
    from base64 import b64encode  # 用來將 PDF 轉成 base64 字串以便 HTML 下載
    from IPython.display import display, HTML  # 用於 Jupyter Notebook 內嵌 HTML
    import io  # 用於圖片轉為記憶體檔案

    # 顯示 Python 的棄用警告（有助於除錯）
    warnings.simplefilter('default', DeprecationWarning)

    # 建立 PDF 文件物件
    pdf = FPDF()
    pdf.add_page()  # 加入一頁

    # 註冊並使用中文字型
    pdf.add_font('NotoSansCJKtc', '', font_path, uni=True)  # Regular
    pdf.set_font('NotoSansCJKtc', size=font_size)  # 預設字型
    pdf.add_font('NotoSansCJKtc', 'B', 'NotoSansCJKtc-Bold.otf')  # Bold
    pdf.set_font('NotoSansCJKtc', style='B', size=font_size)  # 設定為粗體

    # ===== 圖片頁首處理 =====
    image_buffer = io.BytesIO()  # 建立記憶體檔案物件
    image.save(image_buffer, format="PNG")  # 將 PIL 圖片存為 PNG 格式
    image_buffer.seek(0)  # 將檔案指標移至開頭
    pdf.image(image_buffer, x=10, y=20, w=pdf.w - 20)  # 將圖片插入 PDF 頁面
    pdf.ln(100)  # 留空行距避免文字與圖片重疊

    # ===== 處理後續文字內容 =====
    pdf.add_page()  # 加入一頁
    pdf.set_font('NotoSansCJKtc', '', font_size)  # 恢復為 regular 字型
    safe_width = pdf.w - 2 * pdf.l_margin  # 可用文字寬度（扣掉左右邊界）

    # 將輸入文字逐行處理
    for line in text.split("\n"):
        line = line.rstrip()  # 移除右邊空白

        if not line:  # 空行：換段落
            pdf.ln(font_size//2 + 2)

        elif line.startswith('### '):  # 標題格式
            pdf.set_font('NotoSansCJKtc', size=font_size+4, style='B')
            pdf.multi_cell(safe_width, font_size+10, line[4:])  # 切除 ###
            pdf.set_font('NotoSansCJKtc', size=font_size)

        elif line.startswith('**') and line.endswith('**'):  # 粗體行
            pdf.set_font('NotoSansCJKtc', size=font_size+2, style='B')
            pdf.multi_cell(safe_width, font_size+6, line.replace('**',''))
            pdf.set_font('NotoSansCJKtc', size=font_size)

        elif line.startswith('*'):  # 列表項目
            pdf.set_x(pdf.l_margin + 5)  # 稍微縮排
            pdf.multi_cell(safe_width-10, font_size+2, line[1:].strip())

        else:  # 一般段落
            if '**' in line:
                segments = []
                while '**' in line:
                    pre, rest = line.split('**', 1)
                    if '**' not in rest:
                        segments.append((pre + '**' + rest, False))  # 沒有配對，當作普通字
                        break
                    bold, line = rest.split('**', 1)
                    segments.append((pre, False))
                    segments.append((bold, True))
                segments.append((line, False))

                line_height = font_size + 1
                max_width = safe_width
                curr_width = 0
                for seg_text, is_bold in segments:
                    seg_text = seg_text.strip()
                    if not seg_text:
                        continue
                    pdf.set_font('NotoSansCJKtc', style='B' if is_bold else '', size=font_size)
                    # 中文計算寬度
                    text_width = pdf.get_string_width(seg_text)
                    # 斷行（超出就跳行）
                    if curr_width + text_width > max_width:
                        pdf.ln(line_height)
                        pdf.ln(line[curr_width+curr_width,:])
                        pdf.set_x(pdf.l_margin + 5)
                        curr_width = 0
                    pdf.cell(text_width, line_height, seg_text, ln=0, align='L')
                    curr_width += text_width
                pdf.ln(line_height)
                #     text_width = pdf.get_string_width(seg_text)
                #     pdf.cell(text_width, line_height, seg_text, ln=0, align='L')
                # pdf.ln(line_height)  # 一整行結束再換行



                # for seg_text, is_bold in segments:
                #     seg_text = seg_text.strip()
                #     if not seg_text:
                #        continue
                #     pdf.set_font('NotoSansCJKtc', style='B' if is_bold else '', size=font_size)
                #     pdf.set_x(pdf.l_margin + 5)  # 稍微縮排
                #     pdf.multi_cell(safe_width - 20, font_size + 2, seg_text, align='L')
            else:
                pdf.set_font('NotoSansCJKtc', style='', size=font_size)
                pdf.set_x(pdf.l_margin + 5)  # 稍微縮排
                pdf.multi_cell(safe_width, font_size + 2, line, align='L')
            # pdf.set_x(pdf.l_margin + 5)
            # pdf.multi_cell(safe_width-20, font_size+2, line, align='L')

    # 將 PDF 轉為 byte 並編碼成 base64 字串
    pdf_bytes = pdf.output()
    base64_pdf = b64encode(pdf_bytes).decode("utf-8")

    # 回傳 HTML 的下載連結
    html_download = f'<a download="{filename}" href="data:application/pdf;base64,{base64_pdf}">下載 PDF</a>'
    return html_download


--2025-06-03 02:19:04--  https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-03 02:19:04--  https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-03 02:19:04--  https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.19

In [ ]:
#備份
# 設定 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")

    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題", placeholder="東京鐵塔、京都櫻花")
            preference = gr.Textbox(label="旅遊偏好", placeholder="自然景觀/人文藝術/美食/購物")
            budget = gr.Textbox(label="預算（可選）", placeholder="例如：2萬元以內")
            days = gr.Textbox(label="天數（可選）", placeholder="例如：5天4夜")
            group = gr.Textbox(label="人數/身份（可選）", placeholder="2人情侶/全家出遊")
            transport = gr.Textbox(label="交通方式（可選）", placeholder="自駕/大眾運輸")
            style_choice = gr.Radio(
                choices=list(STYLE_PRESETS.keys()),
                value=DEFAULT_STYLE_KEY,
                label="圖像風格（請選擇一項）"
            )
            #pdf_name_input = gr.Textbox(label="PDF 檔名", placeholder="例如 ai_travel_plan.pdf") ##
        with gr.Row():
            submit = gr.Button("生成旅遊建議、圖像、地圖與 PDF")
        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示") # ⬅ 新增地圖顯示欄位
        pdf_download = gr.HTML(label="下載 PDF")
        error_output = gr.Textbox(label="錯誤訊息", lines=3)

        def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice, pdf_name_input="cd.pdf"):
            try:
                # location="大阪"
                # preference="美食"
                # budget = "2萬"
                # days = "5天4夜"
                # group = "1人"
                # transport= "自駕"
                user_prompt = (
                f"請根據以下資訊規劃個人化旅遊建議，包含路線/景點/活動。\n"
                f"地點：{location}\n"
                f"旅遊偏好：{preference}\n"
                f"預算：{budget}\n"
                f"天數：{days}\n"
                f"人數或身份：{group}\n"
                f"交通方式：{transport}\n"
                f"請條列化建議內容。"
                )
                # 1) LLM 文字建議
                llm_result, _ = llm_reply(user_prompt)

                # 2) 生成封面圖
                image_result = generate_cover_image(location, style_choice)

                # 3) AI Agents (Reflection + Planning)
                agent_result = agent_plan_route(location, preference, budget, days, group, transport)

                # 4) 用你的 generate_map_html 函數生成地圖
                map_html = generate_map_html(location)  # ⬅ 加入地圖

                # 5) PDF
                pdf_text = llm_result + agent_result
                PP = generate_and_display_pdf(image=image_result ,text=pdf_text, filename="喵.pdf", font_size=14, width=800, height=400)

                return llm_result, image_result, agent_result, map_html, PP
            except Exception as e:
                # 發生錯誤時回傳錯誤訊息，其它欄位設為空或 None
                return "123", None, "", "", None, str(e)



        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice],
            outputs=[llm_output, image_output, agent_output, map_display, pdf_download]
        )

    with gr.Tab("互動式旅遊聊天機器人"):
        chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")
        chat_input = gr.Textbox(label="請輸入你的問題", placeholder="請問大阪春天有什麼活動？")
        chat_submit = gr.Button("發送")
        chat_state = gr.State([])  # List of messages

        def chat_ai(user_msg, chat_history):
            reply, new_history = llm_reply(user_msg, chat_history)
            chat_history.append((user_msg, reply))
            return chat_history, new_history

        chat_submit.click(
            fn=chat_ai,
            inputs=[chat_input, chat_state],
            outputs=[chatbox, chat_state]
        )

    demo.launch()
#llm_result, image_result, agent_result, map_html, pdf_path = ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 8 arguments for function <function ai_travel_assistant_all at 0x7b4508417100>, received 7.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 8 arguments for function <function ai_travel_assistant_all at 0x7b4508417100>, received 7.
  warnings.warn(
<ipython-input-99-abf55bf0eed0>:81: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://97bc62b3716c9bbdd1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Hugging Face 簡易圖片生成（測試版）


In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO

# ✅ 設定你的 Hugging Face Token
HF_TOKEN = input("請輸入 Hugging Face Token (hf- 開頭)：").strip()

# ✅ 使用支援 API 的模型
HF_SD_MODEL = "stabilityai/stable-diffusion-2"
api_url = f"https://api-inference.huggingface.co/models/{HF_SD_MODEL}"

# ✅ 要畫的圖片描述
prompt = "富士山不僅是日本的象徵，亦是著名的旅遊景點和登山勝地"

# ✅ 發送請求
headers = {"Authorization": f"Bearer {HF_TOKEN}"}
payload = {"inputs": prompt}

response = requests.post(api_url, headers=headers, json=payload)
if response.status_code == 200:
    image_bytes = response.content
    image = Image.open(BytesIO(image_bytes))
    image.show()
else:
    print(f"❌ API 錯誤！狀態碼: {response.status_code}")
    print(response.text)

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install diffusers accelerate transformers safetensors

import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "stabilityai/stable-diffusion-2-1"  # 改用 2-1，支援度高
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# 渲染圖像
prompt = "富士山不僅是日本的象徵，亦是著名的旅遊景點和登山勝地"
image = pipe(prompt).images[0]

# 顯示圖片
image.show()

# 存檔
image.save("fuji_mountain.png")


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")
    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題", placeholder="東京鐵塔、京都櫻花")
            preference = gr.Textbox(label="旅遊偏好", placeholder="自然景觀/人文藝術/美食/購物")
            budget = gr.Textbox(label="預算（可選）", placeholder="例如：2萬元以內")
            days = gr.Textbox(label="天數（可選）", placeholder="例如：5天4夜")
            group = gr.Textbox(label="人數/身份（可選）", placeholder="2人情侶/全家出遊")
            transport = gr.Textbox(label="交通方式（可選）", placeholder="自駕/大眾運輸")
            style_choice = gr.Radio(
                choices=list(STYLE_PRESETS.keys()),
                label="圖像風格（請選擇一項）",
                value="手繪風"
            )
        with gr.Row():
            submit = gr.Button("生成旅遊建議、圖像、地圖與 AI Agents 行程")
        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示")  # ⬅ 新增地圖顯示欄位

        def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice):
            # LLM推薦
            user_prompt = (
                f"請根據以下資訊規劃個人化旅遊建議，包含路線/景點/活動。\n"
                f"地點：{location}\n"
                f"旅遊偏好：{preference}\n"
                f"預算：{budget}\n"
                f"天數：{days}\n"
                f"人數或身份：{group}\n"
                f"交通方式：{transport}\n"
                f"請條列化建議內容。"
            )
            llm_result, _ = llm_reply(user_prompt)
            image_result = generate_cover_image(location, style_choice)

            # AI Agents (Reflection + Planning)
            agent_result = agent_plan_route(location, preference, budget, days, group, transport)

            # 用你的 generate_map_html 函數生成地圖
            map_html = generate_map_html(location)  # ⬅ 加入地圖

            return llm_result, image_result, agent_result, map_html

        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice],
            outputs=[llm_output, image_output, agent_output, map_display]   # ⬅ 對齊四個輸出
        )

    with gr.Tab("互動式旅遊聊天機器人"):
        chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")
        chat_input = gr.Textbox(label="請輸入你的問題", placeholder="請問大阪春天有什麼活動？")
        chat_submit = gr.Button("發送")
        # 保持聊天歷史（用state）
        chat_state = gr.State([])  # List of messages

        def chat_ai(user_msg, chat_history):
            # chat_history 格式：[{"role":"user/assistant","content":...}, ...]
            reply, new_history = llm_reply(user_msg, chat_history)
            chat_history.append((user_msg, reply))
            return chat_history, new_history

        chat_submit.click(
            fn=chat_ai,
            inputs=[chat_input, chat_state],
            outputs=[chatbox, chat_state]
        )

demo.launch()

<ipython-input-35-4b8e0fd79a9e>:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://77ad6cefeda822dbc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from fpdf import FPDF
pdf = FPDF()
pdf.add_page()
pdf.add_font('SourceHan', '', 'SourceHanSansTC-Regular.otf', uni=True)
pdf.set_font('SourceHan', size=28)
pdf.cell(0, 20, "哈囉！這是繁體中文的 PDF 範例。歡迎使用 fpdf2！")
pdf.output("中文PDF範例.pdf")

# Display the PDF in the notebook by embedding it as HTML content:
WIDTH, HEIGHT = 800, 400
from base64 import b64encode
from IPython.display import display, HTML
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<embed src="data:application/pdf;base64,{base64_pdf}" width="{WIDTH}" height="{HEIGHT}" type="application/pdf">'))

# Display a download button:
display(HTML(f'<a download="fpdf2-demo.pdf" href="data:application/pdf;base64,{base64_pdf}">Click to download PDF</a>'))

<ipython-input-15-9296232be9f3>:4: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('SourceHan', '', 'SourceHanSansTC-Regular.otf', uni=True)


FileNotFoundError: TTF Font file not found: SourceHanSansTC-Regular.otf

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw
import base64
import traceback
import io
from fpdf import FPDF

# ===== 模擬功能區 =====

def llm_reply(prompt, history=None):
    return f"[模擬 LLM 回應] 根據輸入：{prompt}", history or []

def generate_cover_image(location, style):
    img = Image.new('RGB', (400, 200), color='lightblue')
    draw = ImageDraw.Draw(img)
    draw.text((10, 80), f"{location} - {style}", fill='black')
    return img

def agent_plan_route(location, preference, budget, days, group, transport):
    return (f"[模擬 AI Agents 規劃]\n"
            f"地點：{location}\n偏好：{preference}\n預算：{budget}\n天數：{days}\n人數：{group}\n交通：{transport}")

def generate_map_html(location):
    return f"<div style='padding:10px;'>這是 {location} 的地圖顯示 (模擬 HTML)</div>"

# ===== 中文 PDF 生成功能（使用支援中文字型） =====

def generate_pdf_base64(text: str, filename: str = "ai_travel_plan.pdf") -> str:
    from fpdf import FPDF
    import io
    import base64

    pdf = FPDF()
    pdf.add_page()

    # 使用支援中文字型的 Noto 字體
    font_path = "/content/NotoSansCJKtc-Regular.ttf"
    pdf.add_font('Noto', '', font_path, uni=True)
    pdf.set_font('Noto', '', 14)

    # 定義安全寬度（頁面寬度減左右邊界）
    safe_width = pdf.w - 2 * pdf.l_margin

    # 處理行列與空行
    for line in text.split("\n"):
        line = line.strip()
        if not line:
            pdf.ln(5)  # 空行直接換行
            continue

        # 強制斷行過長字串避免爆掉
        while len(line) > 0:
            chunk = line[:100]  # 每次處理 100 個字以內，避免爆長中文字串
            pdf.multi_cell(safe_width, 10, chunk)
            line = line[100:]

    # 儲存為 PDF
    buffer = io.BytesIO()
    pdf.output(buffer)
    base64_pdf = base64.b64encode(buffer.getvalue()).decode("utf-8")

    # 產生下載連結 HTML
    return f'''
    <a download="{filename}"
       href="data:application/pdf;base64,{base64_pdf}"
       target="_blank"
       style="font-size:18px;color:blue;">
       📄 點我下載旅遊 PDF
    </a>'''


# ===== 主整合流程 =====

def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice, pdf_name_input):
    try:
        llm_result, _ = llm_reply(
            f"請根據以下資訊規劃個人化旅遊建議：地點：{location}；偏好：{preference}；預算：{budget}；天數：{days}；人數：{group}；交通：{transport}。"
        )
        image_result = generate_cover_image(location, style_choice)
        agent_result = agent_plan_route(location, preference, budget, days, group, transport)
        map_html = generate_map_html(location)

        if not pdf_name_input:
            pdf_name_input = "ai_travel_plan.pdf"
        elif not pdf_name_input.lower().endswith(".pdf"):
            pdf_name_input += ".pdf"

        pdf_html_link = generate_pdf_base64(agent_result, pdf_name_input)

        return llm_result, image_result, agent_result, map_html, pdf_html_link, ""

    except Exception as e:
        error_msg = f"{str(e)}\n\n{traceback.format_exc()}"
        return "", None, "", "", "", error_msg

# ===== Gradio 介面 =====

with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")

    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題")
            preference = gr.Textbox(label="旅遊偏好")
            budget = gr.Textbox(label="預算")
            days = gr.Textbox(label="天數")
            group = gr.Textbox(label="人數/身份")
            transport = gr.Textbox(label="交通方式")
            style_choice = gr.Radio(choices=["風格 A", "風格 B"], value="風格 A", label="圖像風格")
            pdf_name_input = gr.Textbox(label="PDF 檔名", placeholder="例如 ai_travel_plan.pdf")

        submit = gr.Button("生成旅遊建議、圖像、地圖與 PDF")

        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示")
        pdf_download = gr.HTML(label="PDF 下載連結（支援中文）")
        error_output = gr.Textbox(label="錯誤訊息", lines=3)

        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice, pdf_name_input],
            outputs=[llm_output, image_output, agent_output, map_display, pdf_download, error_output]
        )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9aa6a1fc56af32039e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2版


In [ ]:
import gradio as gr
from PIL import Image, ImageDraw
import base64
import traceback
import io
from fpdf import FPDF

# ===== 模擬功能區 =====

def llm_reply(prompt, history=None):
    return f"[模擬 LLM 回應] 根據輸入：{prompt}", history or []

def generate_cover_image(location, style):
    img = Image.new('RGB', (400, 200), color='lightblue')
    draw = ImageDraw.Draw(img)
    draw.text((10, 80), f"{location} - {style}", fill='black')
    return img

def agent_plan_route(location, preference, budget, days, group, transport):
    return (f"[模擬 AI Agents 規劃]\n"
            f"地點：{location}\n偏好：{preference}\n預算：{budget}\n天數：{days}\n人數：{group}\n交通：{transport}")

def generate_map_html(location):
    return f"<div style='padding:10px;'>這是 {location} 的地圖顯示 (模擬 HTML)</div>"

# ===== 中文 PDF 生成功能（使用支援中文字型） =====

def generate_pdf_base64_with_image(text: str, image, filename: str = "ai_travel_plan.pdf") -> str:
    from fpdf import FPDF
    import io
    import base64

    pdf = FPDF()

    # 將圖片轉成暫存檔儲存在記憶體
    image_buffer = io.BytesIO()
    image.save(image_buffer, format="PNG")
    image_buffer.seek(0)

    # === 封面頁 ===
    pdf.add_page()
    pdf.image(image_buffer, x=10, y=20, w=pdf.w - 20)  # 寬度填滿頁面
    pdf.set_font("Arial", size=16)
    pdf.ln(140)  # 推文字往下
    pdf.cell(0, 10, "個人化 AI 旅遊推薦封面", ln=True, align="C")

    # === 行程頁 ===
    pdf.add_page()

    # 設定中文字型
    font_path = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"
    pdf.add_font('Noto', '', font_path, uni=True)
    pdf.set_font('Noto', '', 14)
    safe_width = pdf.w - 2 * pdf.l_margin

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            pdf.ln(5)
            continue
        while len(line) > 0:
            chunk = line[:100]
            pdf.multi_cell(safe_width, 10, chunk)
            line = line[100:]

    # 將 PDF 存到記憶體並轉 base64
    buffer = io.BytesIO()
    pdf.output(buffer)
    base64_pdf = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return f'''
    <a download="{filename}"
       href="data:application/pdf;base64,{base64_pdf}"
       target="_blank"
       style="font-size:18px;color:blue;">
       📄 點我下載旅遊 PDF（含封面圖）
    </a>'''



# ===== 主整合流程 =====

def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice, pdf_name_input):
    try:
        llm_result, _ = llm_reply(
            f"請根據以下資訊規劃個人化旅遊建議：地點：{location}；偏好：{preference}；預算：{budget}；天數：{days}；人數：{group}；交通：{transport}。"
        )
        image_result = generate_cover_image(location, style_choice)
        agent_result = agent_plan_route(location, preference, budget, days, group, transport)
        map_html = generate_map_html(location)

        if not pdf_name_input:
            pdf_name_input = "ai_travel_plan.pdf"
        elif not pdf_name_input.lower().endswith(".pdf"):
            pdf_name_input += ".pdf"

        pdf_html_link = generate_pdf_base64(agent_result, pdf_name_input)

        return llm_result, image_result, agent_result, map_html, pdf_html_link, ""

    except Exception as e:
        error_msg = f"{str(e)}\n\n{traceback.format_exc()}"
        return "", None, "", "", "", error_msg

# ===== Gradio 介面 =====

with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")

    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題")
            preference = gr.Textbox(label="旅遊偏好")
            budget = gr.Textbox(label="預算")
            days = gr.Textbox(label="天數")
            group = gr.Textbox(label="人數/身份")
            transport = gr.Textbox(label="交通方式")
            style_choice = gr.Radio(choices=["風格 A", "風格 B"], value="風格 A", label="圖像風格")
            pdf_name_input = gr.Textbox(label="PDF 檔名", placeholder="例如 ai_travel_plan.pdf")

        submit = gr.Button("生成旅遊建議、圖像、地圖與 PDF")

        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示")
        pdf_download = gr.HTML(label="PDF 下載連結（支援中文）")
        error_output = gr.Textbox(label="錯誤訊息", lines=3)

        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice, pdf_name_input],
            outputs=[llm_output, image_output, agent_output, map_display, pdf_download, error_output]
        )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9aa6a1fc56af32039e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🌏 AI旅遊嚮導：結合圖文生成的個人化旅遊推薦系統")
    with gr.Tab("個人化旅遊推薦"):
        with gr.Row():
            location = gr.Textbox(label="旅遊地點/主題", placeholder="東京鐵塔、京都櫻花")
            preference = gr.Textbox(label="旅遊偏好", placeholder="自然景觀/人文藝術/美食/購物")
            budget = gr.Textbox(label="預算（可選）", placeholder="例如：2萬元以內")
            days = gr.Textbox(label="天數（可選）", placeholder="例如：5天4夜")
            group = gr.Textbox(label="人數/身份（可選）", placeholder="2人情侶/全家出遊")
            transport = gr.Textbox(label="交通方式（可選）", placeholder="自駕/大眾運輸")
            style_choice = gr.Radio(
                choices=list(STYLE_PRESETS.keys()),
                label="圖像風格（請選擇一項）",
                value="手繪風"
            )
        with gr.Row():
            submit = gr.Button("生成旅遊建議、圖像、地圖與 AI Agents 行程")
        llm_output = gr.Textbox(label="旅遊建議（RAG+LLM推薦景點）", lines=5)
        image_output = gr.Image(label="專屬旅遊場景圖片")
        agent_output = gr.Textbox(label="AI Agents 行程規劃 (Reflection & Planning)", lines=10)
        map_display = gr.HTML(label="地圖顯示")  # ⬅ 新增地圖顯示欄位

        def ai_travel_assistant_all(location, preference, budget, days, group, transport, style_choice):
            # LLM推薦
            user_prompt = (
                f"請根據以下資訊規劃個人化旅遊建議，包含路線/景點/活動。\n"
                f"地點：{location}\n"
                f"旅遊偏好：{preference}\n"
                f"預算：{budget}\n"
                f"天數：{days}\n"
                f"人數或身份：{group}\n"
                f"交通方式：{transport}\n"
                f"請條列化建議內容。"
            )
            llm_result, _ = llm_reply(user_prompt)
            image_result = generate_cover_image(location, style_choice)

            # AI Agents (Reflection + Planning)
            agent_result = agent_plan_route(location, preference, budget, days, group, transport)

            # 用你的 generate_map_html 函數生成地圖
            map_html = generate_map_html(location)  # ⬅ 加入地圖

            return llm_result, image_result, agent_result, map_html

        submit.click(
            fn=ai_travel_assistant_all,
            inputs=[location, preference, budget, days, group, transport, style_choice],
            outputs=[llm_output, image_output, agent_output, map_display]   # ⬅ 對齊四個輸出
        )

    with gr.Tab("互動式旅遊聊天機器人"):
        chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")
        chat_input = gr.Textbox(label="請輸入你的問題", placeholder="請問大阪春天有什麼活動？")
        chat_submit = gr.Button("發送")
        # 保持聊天歷史（用state）
        chat_state = gr.State([])  # List of messages

        def chat_ai(user_msg, chat_history):
            # chat_history 格式：[{"role":"user/assistant","content":...}, ...]
            reply, new_history = llm_reply(user_msg, chat_history)
            chat_history.append((user_msg, reply))
            return chat_history, new_history

        chat_submit.click(
            fn=chat_ai,
            inputs=[chat_input, chat_state],
            outputs=[chatbox, chat_state]
        )

demo.launch()

<ipython-input-36-4b8e0fd79a9e>:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="旅遊小助手：自由提問、推薦、查詢、規劃皆可")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5266cd61ef406a1b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# -*- coding: utf-8 -*-
# 下載 Noto Sans CJK 字型
!wget -O NotoSansCJKtc-Regular.ttf "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf"
!wget -O NotoSansCJKtc-Bold.otf "https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Bold.otf"

from fpdf import FPDF
from base64 import b64encode
from IPython.display import display, HTML
import os
#import keyboard

# print("按下空白鍵以繼續...")
# keyboard.wait('space')
# print("繼續執行程式")

def generate_and_display_pdf(text, filename="大阪旅遊規劃.pdf", font_size=14, width=800, height=400, font_path="NotoSansCJKtc-Regular.ttf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font('NotoSansCJKtc', '', font_path, uni=True)
    pdf.set_font('NotoSansCJKtc', size=font_size)
    pdf.add_font('NotoSansCJKtc', 'B', 'NotoSansCJKtc-Bold.otf')
    pdf.set_font('NotoSansCJKtc', style='B', size=font_size)

    # ===== 新增首頁圖像 =====
    # if cover_image_path is not None:
    #     # 假設圖片寬度預設190mm，高度等比例（A4寬度為210mm，左右邊界各10mm）
    #     pdf.image(cover_image_path, x=10, y=20, w=190)
    #     pdf.ln(120)  # 圖片後推進頁面
    #     # 可視需求加上標題或自訂文字
    #     pdf.set_font('NotoSansCJKtc', 'B', font_size+8)
    #     pdf.cell(0, 20, "大阪旅遊專屬規劃", ln=True, align="C")
    #     pdf.add_page()  # 新增一頁作為內容頁

    # ===== 內容頁 =====


    # 適合 A4 寬度
    safe_width = pdf.w - 2 * pdf.l_margin
    # 多行多段落處理
    for line in text.split("\n"):
        print(line)
        #key = input("請按空白鍵後 Enter 鍵以繼續：")
        line = line.rstrip()
        if not line:
            pdf.ln(font_size//2 + 2)  # 空行變段落
        elif line.startswith('### '):
            pdf.set_font('NotoSansCJKtc', size=font_size+4, style='B')
            pdf.multi_cell(safe_width, font_size+10, line[4:])
            pdf.set_font('NotoSansCJKtc', size=font_size)
        elif line.startswith('**') and line.endswith('**'):
            pdf.set_font('NotoSansCJKtc', size=font_size, style='B')
            pdf.multi_cell(safe_width, font_size+6, line.replace('**',''))
            pdf.set_font('NotoSansCJKtc', size=font_size)
        elif line.startswith('*'):
            pdf.set_x(pdf.l_margin + 5)
            pdf.cell(5, font_size+2, '•')
            pdf.multi_cell(safe_width-10, font_size+2, line[1:].strip())
        else:
            pdf.set_x(pdf.l_margin + 5)
            pdf.multi_cell(safe_width-20,font_size+2, line, align='L')
    # 輸出 PDF
    #pdf_bytes = pdf.output(dest="S")
    pdf_bytes = pdf.output()
    base64_pdf = b64encode(pdf_bytes).decode("utf-8")
    #html_preview = f'<embed src="data:application/pdf;base64,{base64_pdf}" width="{width}" height="{height}" type="application/pdf">'
    html_download = f'<a download="{filename}" href="data:application/pdf;base64,{base64_pdf}">下載 PDF</a>'
    display(HTML(html_download))

# 測試用內容
text = '''大阪旅遊規劃

**自動推斷時節/季節：**
根據大阪的氣候，推斷最佳旅遊時節為秋季（9月至11月）或春季（3月至5月），避免旅遊旺季和極端天氣。因此，本行程規劃於5月進行。

**完整行程規劃：**

### 第1天：抵達大阪

* **路線：**
  - 桃園國際機場 → 大阪關西國際機場（預計飛行時間約2小時）
  - 租車（自駕）前往大阪市區
* **住宿：** 大阪市區酒店（預訂經濟型酒店，約$1,500新台幣/晚）
* **餐飲：**
  - 午餐：大阪燒（約$300新台幣）
  - 晚餐：豬排飯（約$400新台幣）
* **景點：** 道頓堀地區自由散步

### 第2天：大阪市區觀光

* **路線：**
  - 上午：大阪城（約$200新台幣）
  - 下午：大阪環球影城（約$2,500新台幣）
* **餐飲：**
  - 午餐：大阪市區午餐（約$400新台幣）
  - 晚餐：壽司（約$500新台幣）
'''

# 直接呼叫生成&預覽
generate_and_display_pdf(text)


--2025-06-02 07:02:21--  https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-02 07:02:22--  https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf [following]
--2025-06-02 07:02:22--  https://raw.githubusercontent.com/notofonts/noto-cjk/main/Sans/OTF/TraditionalChinese/NotoSansCJKtc-Regular.otf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 18

<ipython-input-24-6e1a6d1e20ec>:19: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('NotoSansCJKtc', '', font_path, uni=True)


大阪旅遊規劃

**自動推斷時節/季節：**
根據大阪的氣候，推斷最佳旅遊時節為秋季（9月至11月）或春季（3月至5月），避免旅遊旺季和極端天氣。因此，本行程規劃於5月進行。

**完整行程規劃：**

### 第1天：抵達大阪

* **路線：**
  - 桃園國際機場 → 大阪關西國際機場（預計飛行時間約2小時）
  - 租車（自駕）前往大阪市區
* **住宿：** 大阪市區酒店（預訂經濟型酒店，約$1,500新台幣/晚）
* **餐飲：**
  - 午餐：大阪燒（約$300新台幣）
  - 晚餐：豬排飯（約$400新台幣）
* **景點：** 道頓堀地區自由散步

### 第2天：大阪市區觀光

* **路線：**
  - 上午：大阪城（約$200新台幣）
  - 下午：大阪環球影城（約$2,500新台幣）
* **餐飲：**
  - 午餐：大阪市區午餐（約$400新台幣）
  - 晚餐：壽司（約$500新台幣）



In [ ]:
大阪旅遊規劃

**自動推斷時節/季節：**
根據大阪的氣候，推斷最佳旅遊時節為秋季（9月至11月）或春季（3月至5月），避免旅遊旺季和極端天氣。因此，本行程規劃於5月進行。

**完整行程規劃：**

### 第1天：抵達大阪

* **路線：**
  - 桃園國際機場 → 大阪關西國際機場（預計飛行時間約2小時）
  - 租車（自駕）前往大阪市區
* **住宿：** 大阪市區酒店（預訂經濟型酒店，約$1,500新台幣/晚）
* **餐飲：**
  - 午餐：大阪燒（約$300新台幣）
  - 晚餐：豬排飯（約$400新台幣）
* **景點：** 道頓堀地區自由散步

### 第2天：大阪市區觀光

* **路線：**
  - 上午：大阪城（約$200新台幣）
  - 下午：大阪環球影城（約$2,500新台幣）
* **餐飲：**
  - 午餐：大阪市區午餐（約$400新台幣）
  - 晚餐：壽司（約$500新台幣）
